## PROYETO PILOTO AUTONOMA DESERCION

Luego de revisar varias veces la data, se decide empezar por la tabla con mayor detalle que es EstudianteAsignatura, que tiene información no solo de las asignaturas que el estudiante ha matriculado en un periodo, sino también qué grupos ha tomado de dicha asigantura e incluso qué docentes dictaron clase en cada grupo.

La tabla base del proyecto es EstudianteSemestre, que es la que indica si el estudiante tomó asignaturas en ese semestre o no.   Llegaremos posteriormente a ella.

Usaremos solo una conexión a la base de datos.  Solo ejecutar el siguiente codigo una vez.

Para queries solo se requiere lo siguiente y el resultado queda en el dataframe
1. query = 'SELECT ...'
2. df = pd.read_sql(query, cnx)

Para DML se requiere:
1. crear un cursor = cnx.cursor()
2. query = 'update...'
3. cursor.execute(query)
4. cnx.commit()
5. cursor.close()

In [18]:
from sqlalchemy import create_engine
import pandas as pd

usuario = 'uaopiloto'
contraseña = 'ap'
host = 'localhost'  # Por ejemplo, 'localhost'
base_de_datos = 'UAOPiloto'

# Crea una cadena de conexión
cadena_conexion = f'mysql+mysqlconnector://{usuario}:{contraseña}@{host}/{base_de_datos}'
conexionMySQL = create_engine(cadena_conexion)


# df.to_sql(nombre_de_tabla, motor, if_exists='replace', index=False)

# import mysql.connector
# import pandas as pd

# # Conectarse a la base de datos 
# cnx = mysql.connector.connect(user='uaopiloto', password='ap',
#                               host='localhost',
#                               database='UAOPiloto')

ALGORITMO e HIPOTESIS

1. creamos PERIODOS a partir de los distintos períodos en AXM y que nos queden ordenados, y en una lista
pues luego vamos a confirmar si luego de dos períodos estos estudiantes han dejado de matricular.
1. creamos DATOS a partir de Matrícula.
2. adicionamos la columna LABEL que debe ser 1 si luego de 2 periodos,  el estudiante dejó de matricular mínimo 2 periodos consecutivos o más y no fue que se graduó.  De lo contrario debe ser 0.

BASE:  puede ser que nos paramos al final de un semestre, o al prinpcio de un semestre.    Este segundo caso nos permitiria jugar con 202302.  Asi que escogemos asumir que estamos iniciando un semestre y por tanto no tenemos notas del semestre actual.

HIPOTESIS

1.  promedio acumulado de 4 semestres anteriores
2.  promedio por semestre de 4 semestres anteriores
3.  cantidad de asignaturas que tomó el semestre actual y 4 semestres hacia atras
4.  cantidad de créditos que tomó semestre actual y 4 semestres anteriores
5.  cantidad de créditos que perdió 4 semestres anteriores
6.  cantidad de asignaturas que perdió 4 semestres anteriores
7.  si asistió a actividades extracurriculares 4 semestres anteriores
8.  que tan heterogenoes fueron los grupos que toma este semestre y 4 anteriores (desvStd edad, relación F:M)
9.  tuvo mora en 4 semestres anteriores
10. docentes - edad promedio docentes este semestre y anteriores
11. asignaturas que tomó y sus notas en 4 semestres anteriores (son un jurgo de variables)


Algoritmo Paso 1:  creamos PERIODOS

In [34]:
# Traemos los PERIODOS
query = """
SELECT distinct PERIODO 
FROM EstudianteAsignatura
ORDER BY PERIODO"""

# Leer resultados en un DataFrame y lo convertimos en una lsita
PERIODOS = pd.read_sql_query(query, conexionMySQL)
lPeriodos = PERIODOS['PERIODO'].to_list()

# Treamos una lista de estudiantes
query = """
SELECT distinct id_estudiante 
FROM EstudianteAsignatura
ORDER BY id_estudiante"""

# Leer resultados en un DataFrame y lo convertimos en una lsita
ESTUDIANTES = pd.read_sql_query(query, conexionMySQL)
lEstudiantes = ESTUDIANTES['id_estudiante'].to_list()



Algoritmo Paso 2:  cremos DATOS a partir EstudianteAsignatura(id_estudiante, periodo)

In [30]:
# Hacer una consulta SQL
query = """
SELECT distinct id_estudiante, periodo 
FROM EstudianteAsignatura"""

# Leer resultados en un DataFrame
MATRICULAS = pd.read_sql_query(query, conexionMySQL)
MATRICULAS.shape

(832, 2)

Algoritmo paso 3:  crear columna LABEL y que sea 1 si dado una flia en matricula, NO hay fila en matrícula durante 2 periodos consecutivos posteriores.

In [38]:
for e in range(0, len(lEstudiantes)):

    estaEstudiando = False

    for p in range(0, len(lPeriodos)):

        if ((MATRICULAS['id_estudiante'] == lEstudiantes[e]) & (MATRICULAS['periodo'] == lPeriodos[p])).any():
            estaEstudiando = True
            print('analizando ',lEstudiantes[e], lPeriodos[p] )
        
        # miramos si dos semestres siguiente esta estudiando.  Si no lo está haciendo, marcamos como desertor
        if estaEstudiando:

            semestre_1 = ((MATRICULAS['id_estudiante'] == lEstudiantes[e]) & (MATRICULAS['periodo'] == lPeriodos[p+1])).any() 
            semestre_2 = ((MATRICULAS['id_estudiante'] == lEstudiantes[e]) & (MATRICULAS['periodo'] == lPeriodos[p+2])).any() 

            if semestre_1 & semestre_2:
                
                pass 

            else:
                MATRICULAS[((MATRICULAS['id_estudiante'] == lEstudiantes[e]) & (MATRICULAS['periodo'] == lPeriodos[p]))]['LABEL'] = 1

    break
    

analizando  2136689 201803


/var/folders/tx/1tx9w9fj6v78n5f12zkwg_vc0000gn/T/ipykernel_16595/3004866355.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MATRICULAS[((MATRICULAS['id_estudiante'] == lEstudiantes[e]) & (MATRICULAS['periodo'] == lPeriodos[p]))]['LABEL'] = 1


IndexError: list index out of range